In [1]:
import os

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

import scipy.interpolate as interpolate
import scipy.constants as constants

In [2]:
%matplotlib widget

In [3]:
base_path = '/Users/manifestation/Stanford/beads/comsol_models/new_trap_efield/near_field/data/'
# base_path = '/USERS/NADAV/PATH/TO/ANALYSIS/FOLDER/'
base_path = r'/home/nadavp/tmp/'
fname = 'new_trap_attractor_shield_potential_efield.txt'


positions_to_sample = np.array([
                                [13.9e-6, 0.0e-6, -15.23e-6], \
                                [30e-6, 0.0e-6, -15.23e-6], \
                                [40e-6, 0.0e-6, -15.23e-6], \
                                [50e-6, 0.0e-6, -15.23e-6], \
                               ])

dipole_units = constants.elementary_charge * 1.0e-6
dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units


In [4]:
#############################################################
######  LOAD DATA FROM COMSOL OUTPUT TO NUMPY ARRAYS  #######
#############################################################

path = os.path.join(base_path, fname)

with open(path, 'r') as f:
    lines  =  f.readlines()

for lineind, line in enumerate(lines):
    if 'Grid' in line:
        grid_ind = lineind + 1
    elif 'V (V)' in line:
        voltage_ind = lineind + 1
    elif 'es.Ex (V/m)' in line:
        ex_ind = lineind + 1
    elif 'es.Ey (V/m)' in line:
        ey_ind = lineind + 1
    elif 'es.Ez (V/m)' in line:
        ez_ind = lineind + 1

xvec_str = lines[grid_ind].strip('\n')
yvec_str = lines[grid_ind+1].strip('\n')
zvec_str = lines[grid_ind+2].strip('\n')

xvec = np.array( [float(val) for val in xvec_str.split()] )
yvec = np.array( [float(val) for val in yvec_str.split()] )
zvec = np.array( [float(val) for val in zvec_str.split()] )

lenx = len(xvec)
leny = len(yvec)
lenz = len(zvec)

voltage = np.zeros((lenx, leny, lenz))
efieldx = np.zeros((lenx, leny, lenz))
efieldy = np.zeros((lenx, leny, lenz))
efieldz = np.zeros((lenx, leny, lenz))

for j in range(leny):
    for k in range(lenz):
        voltage_line = lines[voltage_ind + j + k*leny]
        voltage[:,j,k] = np.array( [float(val) for val in voltage_line.strip('\n').split()] ) 

        efieldx_line = lines[ex_ind + j + k*leny]
        efieldx[:,j,k] = np.array( [float(val) for val in efieldx_line.strip('\n').split()] )

        efieldy_line = lines[ey_ind + j + k*leny]
        efieldy[:,j,k] = np.array( [float(val) for val in efieldy_line.strip('\n').split()] )

        efieldz_line = lines[ez_ind + j + k*leny]
        efieldz[:,j,k] = np.array( [float(val) for val in efieldz_line.strip('\n').split()] )





#############################################################
##############  BUILD 3D GRID INTERPOLATORS  ################
#############################################################

voltage_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), voltage)

efieldx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldx)
efieldy_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldy)
efieldz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), efieldz)

In [5]:
#############################################################
############  SIMPLE PLOT AT Z=0 FOR EXAMPLE  ###############
#############################################################

zcoord = 0.0

n_arrow = 9
arrow_xvec = np.linspace(10e-6, 90e-6, n_arrow)
arrow_yvec = np.linspace(-180e-6, 180e-6, n_arrow)
arrow_efieldx = np.zeros((n_arrow,n_arrow))
arrow_efieldy = np.zeros((n_arrow,n_arrow))
for i in range(n_arrow):
    for  j in range(n_arrow):
        arrow_efieldx[i,j] = efieldx_func([arrow_xvec[i], arrow_yvec[j], zcoord])
        arrow_efieldy[i,j] = efieldy_func([arrow_xvec[i], arrow_yvec[j], zcoord])

norm = colors.Normalize(vmin=0, vmax=1)

z_ind = np.argmin(np.abs(zvec - zcoord))

fig, ax = plt.subplots(1,1)

divider = make_axes_locatable(ax)
ax_cb = divider.new_horizontal(size='4%', pad=0.1)
cb1 =  mpl.colorbar.ColorbarBase(ax_cb, cmap=mpl.cm.plasma, \
                                 norm=norm, orientation='vertical')
cb1.set_label('Voltage [V]')
fig.add_axes(ax_cb)

ax.set_title('Voltage at $z=0$ plane\nwith attractor biased to 1 V')

ax.contourf(xvec*1e6, yvec*1e6, voltage[:,:,z_ind].T, 100, cmap='plasma', norm=norm)

quiver = ax.quiver(arrow_xvec*1e6, arrow_yvec*1e6, arrow_efieldx[:,:].T, arrow_efieldy[:,:], \
                   color='w', pivot='mid', scale=1e5, label='Efield')
ax.quiverkey(quiver, X=0.7, Y=0.85, U=1.0e4, label='$10^4$ V/m', labelcolor='w')

ax.set_xlabel('X-coordinate [$\\mu$m]')
ax.set_ylabel('Y-coordinate [$\\mu$m]')
fig.tight_layout()
fig.show()
plt.show()
# fig.savefig('sruface_map_voltage.png')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
charges = 1e-3
force = charges * 1.6e-19 * 400 / 8e-3 * 0.61  # in Newtons
force

4.8799999999999995e-18

In [7]:
grad_mili = 1e6
grad_mili*dipole_moment[0]

1.602176634e-17

In [10]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
for point in positions_to_sample:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [ 1.22692234e-15 -2.96041324e-16 -2.75802579e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 30.     0.   -15.23]
        Efield [V/m] :  [-2267.37095064   -31.10601837  -828.73414718]
Force on dipole [N]  :  [ 1.29930806e-15 -3.14455696e-18 -7.14814877e-16]

    MS Postion [um]  :  [ 40.     0.   -15.23]
        Efield [V/m] :  [-1556.31307815   -42.98086473 -1084.36550179]
Force on dipole [N]  :  [ 9.43004684e-16  3.07722744e-17 -2.22820070e-16]

    MS Postion [um]  :  [ 50.     0.   -15.23]
        Efield [V/m] :  [-1076.59837863   -42.33409451 -1121.51209126]
Force on dipole [N]  :  [ 7.27393329e-16 -6.58035028e-18  1.13438509e-16]



In [11]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
for point in positions_to_sample:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [ 1.22692234e-15 -2.96041324e-16 -2.75802579e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or

    MS Postion [um]  :  [ 30.     0.   -15.23]
        Efield [V/m] :  [-2267.37095064   -31.10601837  -828.73414718]
Force on dipole [N]  :  [ 1.29930806e-15 -3.14455696e-18 -7.14814877e-16]

    MS Postion [um]  :  [ 40.     0.   -15.23]
        Efield [V/m] :  [-1556.31307815   -42.98086473 -1084.36550179]
Force on dipole [N]  :  [ 9.43004684e-16  3.07722744e-17 -2.22820070e-16]

    MS Postion [um]  :  [ 50.     0.   -15.23]
        Efield [V/m] :  [-1076.59837863   -42.33409451 -1121.51209126]
Force on dipole [N]  :  [ 7.27393329e-16 -6.58035028e-18  1.13438509e-16]



In [117]:
dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units
positions_to_sample2 = np.array([[dist_*1e-6, 4.9e-6, -15.23e-6] for dist_ in np.arange(10,50,2.5)])

In [118]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x = []
force_z = []
for point in positions_to_sample2:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x.append(force_on_dipole[0])
    force_z.append(force_on_dipole[2])


    MS dipole [e um] :  [  0. 100.   0.]

    MS Postion [um]  :  [ 10.     4.9  -15.23]
        Efield [V/m] :  [-3402.32609917  -153.15188481  1158.27445391]
Force on dipole [N]  :  [-1.04116542e-15 -1.16592266e-15  1.08941043e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 12.5    4.9  -15.23]
        Efield [V/m] :  [-3787.98674453  -220.47084801  1232.82975698]
Force on dipole [N]  :  [ 4.27584219e-16  9.64373299e-16 -5.95164667e-16]

    MS Postion [um]  :  [ 15.     4.9  -15.23]
        Efield [V/m] :  [-3543.045309      24.2761565    628.07325318]
Force on dipole [N]  :  [-2.19763890e-16 -4.05362387e-16 -1.28769390e-16]

    MS Postion [um]  :  [ 17.5    4.9  -15.23]
        Efield [V/m] :  [-3380.53314773   -13.96582233   266.31494892]
Force on dipole [N]  :  [-1.82767643e-16 -1.32764694e-16 -1.08284477e-16]

    MS Postion [um]  :  [ 20.     4.9  -15.23]
        Efield [V/m] :  [-3132.94748158   -22.65156466   -52.74740141]
Force on dipole [N]  :  [1.09879050e-16 2.64465229e-16 4.21124232e-17]

    MS Postion [um]  :  [ 22.5    4.9  -15.23]
        Efield [V/m] :  [-2875.23860704   -16.78515443  -341.96730238]
Force on dipole [N]  :  [-7.15031094e-17 -6.07150780e-18  1.62677942e-17]

    MS Postion [um]  :  [ 25.     4.9 

In [120]:
force_x = np.array(force_x)
force_z = np.array(force_z)

In [16]:
_,ax = plt.subplots()
ax.scatter(np.arange(10,50,2.5), force_x/10, label='X-backgrounds')
ax.scatter(np.arange(10,50,2.5), force_z/10, label='Z-backgrounds')
ax.legend()
ax.set(ylabel='Force [N]', xlabel=r'x distance [$\mu m$]', title='MS dipole [e um] : '+str(dipole_moment/dipole_units))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'force_x' is not defined

In [137]:
dipole_moment2 = np.array([[0.0, 100.0*np.sin(theta), 100.0*np.cos(theta)] for theta in np.arange(0,2*np.pi,0.2)]) * dipole_units
positions_to_sample3 = np.array([13.9*1e-6, 0.0e-6, -15.23e-6])

In [138]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x2 = []
force_z2 = []
for edm in dipole_moment2:

    sample_efield = np.array([efieldx_func(positions_to_sample3)[0], efieldy_func(positions_to_sample3)[0], efieldz_func(positions_to_sample3)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(positions_to_sample3) * edm[i]
        force_on_dipole[1] += grady_func(positions_to_sample3) * edm[i]
        force_on_dipole[2] += gradz_func(positions_to_sample3) * edm[i]

    print('    MS Postion [um]  : ', positions_to_sample3*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x2.append(force_on_dipole[0])
    force_z2.append(force_on_dipole[2])


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.72888165e-15 -4.42228469e-17 -9.76181112e-16]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.75960022e-15  4.42464201e-17 -9.86127840e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.68030223e-15  1.30951722e-16 -9.56760762e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.49414905e-15  2.12436392e-16 -8.89250652e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-2.20856202e-15  2.85451893e-16 -7.86288925e-16]

    MS Postion [um]  :  [ 13.9    0.   -15.23]
        Efield [V/m] :  [-3.62617454e+03  6.44146890e-01  7.49210954e+02]
Force on dipole [N]  :  [-1.83492659e-15  3.47087328e-16 -6.51980340e-16]

    MS Postion [um] 

In [132]:
np.argmax(force_x2), np.argmin(force_x2)

(330, 16)

In [130]:
_,ax = plt.subplots()
ax.scatter(np.arange(0,2*np.pi,0.01), force_x2, label='X-backgrounds')
# ax.scatter(np.arange(0,2*np.pi,0.01), dipole_moment2[:,2], label='X-backgrounds')
# ax.scatter(np.arange(0,2*np.pi,0.01), force_z2, label='Z-backgrounds')
ax.legend()
ax.set(ylabel='Force [N]', xlabel=r'Angle [rad]', title='Spinning in YZ plane')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[Text(0, 0.5, 'Force [N]'),
 Text(0.5, 0, 'Angle [rad]'),
 Text(0.5, 1.0, 'Spinning in YZ plane')]

In [139]:
np.sum(force_x2), np.sum(force_z2), np.mean(force_x2), np.mean(force_z2)

(-1.5838231683203737e-15,
 -5.666959708170893e-16,
 -4.949447401001168e-17,
 -1.770924908803404e-17)

In [70]:
len(force_x2)

629

In [9]:
dipole_moment = np.array([100.0, 0.0, 0.0]) * dipole_units
x_att = [50*np.sin(3*t/5000*2*np.pi) for t in range(5000)]
positions_to_sample4 = np.array([[13.9*1e-6, (offset_-4.9)*1e-6, -15.23e-6] for offset_ in x_att])

In [10]:
#############################################################
#######  SAMPLE EFIELD AND DIPOLE/GRADIENT COUPLING  ########
#############################################################

print()
print('    MS dipole [e um] : ', dipole_moment/dipole_units)
print()
force_x4 = []
force_z4 = []
for point in positions_to_sample4:

    sample_efield = np.array([efieldx_func(point)[0], efieldy_func(point)[0], efieldz_func(point)[0]])

    ### Compute:  p (dot) (grad) E
    force_on_dipole = np.zeros(3)
    for i in [0,1,2]:
        gradx = np.gradient(efieldx, np.array([xvec, yvec, zvec])[i], axis=i)
        grady = np.gradient(efieldy, np.array([xvec, yvec, zvec])[i], axis=i)
        gradz = np.gradient(efieldz, np.array([xvec, yvec, zvec])[i], axis=i)
        gradx_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradx)
        grady_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), grady)
        gradz_func = interpolate.RegularGridInterpolator((xvec, yvec, zvec), gradz)
        force_on_dipole[0] += gradx_func(point) * dipole_moment[i]
        force_on_dipole[1] += grady_func(point) * dipole_moment[i]
        force_on_dipole[2] += gradz_func(point) * dipole_moment[i]

    print('    MS Postion [um]  : ', point*1e6)
    print('        Efield [V/m] : ', sample_efield)
    print('Force on dipole [N]  : ', force_on_dipole)
    print()
    force_x4.append(force_on_dipole[0])
    force_z4.append(force_on_dipole[2])


    MS dipole [e um] :  [100.   0.   0.]

    MS Postion [um]  :  [ 13.9   -4.9  -15.23]
        Efield [V/m] :  [-3629.04376377   -32.38996578   745.93786973]
Force on dipole [N]  :  [ 9.17323934e-16  1.74836754e-16 -2.84188688e-15]



/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:17: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/home/analysis_user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dt

    MS Postion [um]  :  [ 13.9         -4.71150489 -15.23      ]
        Efield [V/m] :  [-3627.01507648   -32.22025623   745.26695904]
Force on dipole [N]  :  [ 9.18430770e-16  1.71073316e-16 -2.83949946e-15]

    MS Postion [um]  :  [ 13.9         -4.52301245 -15.23      ]
        Efield [V/m] :  [-3624.98641803   -32.0505491    744.59605789]
Force on dipole [N]  :  [ 9.19537591e-16  1.67309931e-16 -2.83711207e-15]

    MS Postion [um]  :  [ 13.9         -4.33452538 -15.23      ]
        Efield [V/m] :  [-3622.95781725   -31.88084679   743.92517581]
Force on dipole [N]  :  [ 9.20644380e-16  1.63546654e-16 -2.83472475e-15]

    MS Postion [um]  :  [ 13.9         -4.14604634 -15.23      ]
        Efield [V/m] :  [-3620.92930296   -31.71115171   743.25432234]
Force on dipole [N]  :  [ 9.21751122e-16  1.59783537e-16 -2.83233754e-15]

    MS Postion [um]  :  [ 13.9         -3.95757801 -15.23      ]
        Efield [V/m] :  [-3619.04582022   -31.4905149    742.63567494]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9         2.7975821 -15.23     ]
        Efield [V/m] :  [-3774.79189554    70.26226455   702.75288094]
Force on dipole [N]  :  [ 1.13361416e-15 -2.36949041e-16 -2.99668286e-15]

    MS Postion [um]  :  [ 13.9          2.98377535 -15.23      ]
        Efield [V/m] :  [-3773.48395295    59.43222631   718.24506925]
Force on dipole [N]  :  [ 1.10861052e-15 -1.75358000e-16 -3.07756683e-15]

    MS Postion [um]  :  [ 13.9          3.16985655 -15.23      ]
        Efield [V/m] :  [-3776.49178359     9.82578936   760.09962681]
Force on dipole [N]  :  [ 1.18411266e-15  1.67797757e-16 -3.33273422e-15]

    MS Postion [um]  :  [ 13.9          3.35582306 -15.23      ]
        Efield [V/m] :  [-3779.90967264   -43.45232314   804.44585172]
Force on dipole [N]  :  [ 1.26916120e-15  5.37623958e-16 -3.60438622e-15]

    MS Postion [um]  :  [ 13.9          3.54167224 -15.23      ]
        Efield [V/m] :  [-3783.3254052    -96.6968204    848.76409687]
Force on dipole [N]  : 

    MS Postion [um]  :  [ 13.9         10.13196084 -15.23      ]
        Efield [V/m] :  [-3586.61051714    -5.83837296   846.13805788]
Force on dipole [N]  :  [ 7.34605697e-16 -3.90851743e-17 -3.56364417e-15]

    MS Postion [um]  :  [ 13.9         10.31162896 -15.23      ]
        Efield [V/m] :  [-3.58770648e+03 -2.82560911e+00  8.42828628e+02]
Force on dipole [N]  :  [ 7.39354720e-16 -7.80187577e-17 -3.55331909e-15]

    MS Postion [um]  :  [ 13.9         10.49108089 -15.23      ]
        Efield [V/m] :  [-3.58880112e+03  1.83529537e-01  8.39523180e+02]
Force on dipole [N]  :  [ 7.44098029e-16 -1.16905493e-16 -3.54300642e-15]

    MS Postion [um]  :  [ 13.9         10.67031408 -15.23      ]
        Efield [V/m] :  [-3.58989443e+03  3.18900022e+00  8.36221761e+02]
Force on dipole [N]  :  [ 7.48835556e-16 -1.55744828e-16 -3.53270633e-15]

    MS Postion [um]  :  [ 13.9         10.84932598 -15.23      ]
        Efield [V/m] :  [-3590.98638282     6.19076024   832.92441891]
Force on di

...

    MS Postion [um]  :  [ 13.9        -33.57310278 -15.23      ]
        Efield [V/m] :  [-3562.30303991   -65.62121042   747.42960359]
Force on dipole [N]  :  [ 3.11064391e-16  1.87471289e-16 -3.28479522e-15]

    MS Postion [um]  :  [ 13.9        -33.41847777 -15.23      ]
        Efield [V/m] :  [-3560.53016385   -65.60465839   746.35148922]
Force on dipole [N]  :  [ 3.09427873e-16  1.88405712e-16 -3.28430924e-15]

    MS Postion [um]  :  [ 13.9        -33.26344746 -15.23      ]
        Efield [V/m] :  [-3558.75264064   -65.58806297   745.27054885]
Force on dipole [N]  :  [ 3.07787065e-16  1.89342585e-16 -3.28382198e-15]

    MS Postion [um]  :  [ 13.9        -33.10801403 -15.23      ]
        Efield [V/m] :  [-3556.97049554   -65.5714244    744.18679784]
Force on dipole [N]  :  [ 3.06141991e-16  1.90281894e-16 -3.28333346e-15]

    MS Postion [um]  :  [ 13.9        -32.95217971 -15.23      ]
        Efield [V/m] :  [-3555.12244469   -65.58691647   743.11653017]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -27.09423168 -15.23      ]
        Efield [V/m] :  [-3479.73680388   -41.25209809   836.30842586]
Force on dipole [N]  :  [ 4.74309690e-16  6.30786901e-17 -3.97019730e-15]

    MS Postion [um]  :  [ 13.9        -26.92516652 -15.23      ]
        Efield [V/m] :  [-3480.07444076   -40.06418099   836.58600018]
Force on dipole [N]  :  [ 4.77389486e-16  6.19229190e-17 -3.97525932e-15]

    MS Postion [um]  :  [ 13.9        -26.75578833 -15.23      ]
        Efield [V/m] :  [-3480.41270278   -38.87406444   836.86408844]
Force on dipole [N]  :  [ 4.75914121e-16  6.29553420e-17 -3.98627072e-15]

    MS Postion [um]  :  [ 13.9        -26.58609953 -15.23      ]
        Efield [V/m] :  [-3480.75158513   -37.68176536   837.14268668]
Force on dipole [N]  :  [ 4.74436051e-16  6.39896585e-17 -3.99730232e-15]

    MS Postion [um]  :  [ 13.9        -26.41610252 -15.23      ]
        Efield [V/m] :  [-3481.091083     -36.4873007    837.42179094]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -20.11162896 -15.23      ]
        Efield [V/m] :  [-3584.9488044    -19.62957007   741.55677452]
Force on dipole [N]  :  [ 8.08090139e-16 -3.46131200e-17 -2.95127276e-15]

    MS Postion [um]  :  [ 13.9        -19.93196084 -15.23      ]
        Efield [V/m] :  [-3586.11318682   -18.98902768   741.63711571]
Force on dipole [N]  :  [ 8.31316967e-16 -4.23658624e-17 -2.86671575e-15]

    MS Postion [um]  :  [ 13.9        -19.75207908 -15.23      ]
        Efield [V/m] :  [-3586.84433443   -18.4509155    742.81691543]
Force on dipole [N]  :  [ 8.18004954e-16 -4.81024239e-17 -2.85273524e-15]

    MS Postion [um]  :  [ 13.9        -19.57198624 -15.23      ]
        Efield [V/m] :  [-3587.57633999   -17.91217187   743.99809957]
Force on dipole [N]  :  [ 8.04677320e-16 -5.38457169e-17 -2.83873831e-15]

    MS Postion [um]  :  [ 13.9        -19.39168488 -15.23      ]
        Efield [V/m] :  [-3588.30919311   -17.37280446   745.18065136]
Force on dipole [N] 

    MS Postion [um]  :  [ 13.9        -12.78377535 -15.23      ]
        Efield [V/m] :  [-3555.90656063    14.63677138   934.10855911]
Force on dipole [N]  :  [ 5.63305622e-16 -2.76757111e-16 -3.27485540e-15]

    MS Postion [um]  :  [ 13.9       -12.5975821 -15.23     ]
        Efield [V/m] :  [-3556.47976468    18.10944664   933.49987408]
Force on dipole [N]  :  [ 4.86683778e-16 -3.64006572e-16 -3.36182190e-15]

    MS Postion [um]  :  [ 13.9        -12.41127946 -15.23      ]
        Efield [V/m] :  [-3557.05330552    21.58416231   932.89083141]
Force on dipole [N]  :  [ 4.10016913e-16 -4.51307298e-16 -3.44883949e-15]

    MS Postion [um]  :  [ 13.9        -12.22487006 -15.23      ]
        Efield [V/m] :  [-3557.627175      25.060869     932.28143975]
Force on dipole [N]  :  [ 3.33306119e-16 -5.38658046e-16 -3.53590694e-15]

    MS Postion [um]  :  [ 13.9        -12.03835656 -15.23      ]
        Efield [V/m] :  [-3558.20136496    28.5395173    931.67170777]
Force on dipole [N]  : 

    MS Postion [um]  :  [ 13.9         -5.27698755 -15.23      ]
        Efield [V/m] :  [-3633.17714873   -32.80550203   747.39197307]
Force on dipole [N]  :  [ 9.18080510e-16  1.77261360e-16 -2.84206074e-15]

    MS Postion [um]  :  [ 13.9         -5.08849511 -15.23      ]
        Efield [V/m] :  [-3631.09674492   -32.58399486   746.64465658]
Force on dipole [N]  :  [ 9.17166061e-16  1.76970078e-16 -2.84280434e-15]



In [31]:
freq2 = np.arange(0,2500.001,0.01)
fft_z2 = np.zeros(len(freq2))+1e-23
fft_x2 = np.zeros(len(freq2))+1e-23

In [34]:
for freq_ in freq1:
    fft_z2[freq2==freq_] = fft_z[freq1==freq_]
    fft_x2[freq2==freq_] = fft_x[freq1==freq_]

In [23]:
freq1[1], freq1[-1], freq2[1], freq2[-1]

(1.0, 2500.0, 0.01, 2500.0)

In [47]:
def fft_norm(N, fsamp):
    "Factor to normalize FFT to ASD units"
    return np.sqrt(2 / (N * fsamp))
scale = fft_norm(5000, 5000)

fig,ax = plt.subplots(1,2,figsize=(9.2,4), sharey=True)
fft_x = np.abs(np.fft.rfft(force_x4))*scale
fft_z = np.abs(np.fft.rfft(force_z4))*scale
angles_z = np.angle(np.fft.rfft(force_z4))
freq1 = np.fft.rfftfreq(len(force_x4), d=1./5000)

freqs_gravity = 3*np.arange(14)
z_to_x = np.array([fft_z[freq1==freq_]/fft_x[freq1==freq_] for freq_ in freqs_gravity])

freq1 = np.fft.rfftfreq(len(force_x4), d=1./5000)
# ax[0].semilogy(freq1, fft_x, label='x-backgrounds')
ax[0].semilogy(freq2, fft_x2)
ax[0].set_title(r'EDM background model - X force')
ax[0].set_xlabel('Frequency [Hz]', size=11)
ax[1].set_xlabel('Frequency [Hz]', size=11)
ax[0].set_ylabel(r'Force ASD [N/$\sqrt{Hz}$]', size=11)

# ax[1].semilogy(freq1, fft_z, label='z-backgrounds')
ax[1].semilogy(freq2, fft_z2, label='z-backgrounds')
ax[1].set_title(r'EDM background model - Z force')
ax[0].set(xlim=(1,70), ylim=(1e-19,1e-14))
ax[1].set(xlim=(1,70), ylim=(1e-19,1e-14))
# ax[0].legend()
# ax[1].legend()
# ax[1].scatter(freqs_gravity, z_to_x)
# ax[1].set(ylabel='Z to X background ratio', xlabel='frequency [Hz.]')
freqs_gravity = 3*np.arange(14)
[print(freq_, angles_z[freq1==freq_]) for freq_ in freqs_gravity]
fig.savefig("x_dipole_backgrounds_template_v2.pdf", bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

0 [3.14159265]
3 [1.57079633]
6 [3.14159265]
9 [-1.57079633]
12 [1.36785058e-17]
15 [-1.57079633]
18 [-1.77005236e-15]
21 [1.57079633]
24 [-2.29488022e-15]
27 [-1.57079633]
30 [-1.75074754e-15]
33 [1.57079633]
36 [3.14159265]
39 [1.57079633]
